In [1]:
import pandas as pd 
import spacy

2022-12-11 11:09:44.008938: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-11 11:09:44.008970: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-11 11:09:45.543658: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-11 11:09:45.543688: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-11 11:09:45.543709: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (u): /proc/driver/nvidia/version does not exist


In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
def remove_stop_words(text):
    doc = nlp(text)
    txt = []
    for token in doc:
        if token.is_stop:
            continue
        txt.append(token.text)
    return " ".join(txt) 

In [4]:
remove_stop_words("using sklearn pipeline module create a classification pipeline to classify the Data")

'sklearn pipeline module create classification pipeline classify Data'

In [5]:
fake_df = pd.read_csv('../data/Fake.csv')
real_df = pd.read_csv('../data/True.csv')

In [6]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [7]:
real_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [11]:
real_df['label'] = [1]*real_df.shape[0]

In [12]:
fake_df['label'] = [0]*fake_df.shape[0]

In [13]:
df = pd.concat([fake_df, real_df], axis=0)

In [15]:
df.shape

(44898, 5)

In [16]:
df['preprocessed_text'] = df['text'].apply(remove_stop_words)

In [17]:
df.head()

,title,text,subject,date,label,preprocessed_text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0,Donald Trump couldn t wish Americans Happy New...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0,House Intelligence Committee Chairman Devin Nu...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0,"Friday , revealed Milwaukee Sheriff David Clar..."
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0,"Christmas day , Donald Trump announced work ..."
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0,Pope Francis annual Christmas Day message rebu...


In [18]:
x = df['preprocessed_text']
y = df['label']

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x, 
                                                    y, 
                                                    test_size=.1, 
                                                    stratify=y,
                                                    random_state=22,
                                                    shuffle=True)

In [23]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

model = Pipeline([
    ("vec", CountVectorizer(ngram_range=(1, 3))),
    ("model", MultinomialNB())
])

model.fit(X_train, Y_train)

Y_test_pred = model.predict(X_test)

print(classification_report(Y_test, Y_test_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      2348
           1       0.98      0.98      0.98      2142

    accuracy                           0.98      4490
   macro avg       0.98      0.98      0.98      4490
weighted avg       0.98      0.98      0.98      4490

